In [1]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
import numpy as np

import os
import time
import cmath

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Constants and settings
WINDOW_SIZE = 25  # Has to be odd
TARGET_COL = WINDOW_SIZE//2

DTYPE = 'float32'

# cqt related
FFT_BINS = 768 # function of items below
HOP_LENGTH = 128 # Required for good cqt results

# stft values
N_FFT = 1024 # 512 recommended for speech, music typically 2048

# cqt values
BINS_PER_OCTAVE = 12 * 8
FMIN = librosa.note_to_hz('C1')
OCTAVES = 8

In [3]:
# Idea here is that we operate on magnitude, and will just use phase from the original noisy sample

#def combine_target(t):
#    return (t[0:t.shape[0]//2] + 1j * t[t.shape[0]//2:]).reshape(1,(t.shape[0]//2))

def rebuild_fft(output, original_fft):
    vphase = np.vectorize(cmath.phase)
    o_phase = vphase(original_fft)
    mag = output.T
    vrect = np.vectorize(cmath.rect)
    return vrect(mag, o_phase)
    
def get_ft(wav):
    c = librosa.cqt(wav, hop_length=HOP_LENGTH, fmin=FMIN, n_bins=OCTAVES*BINS_PER_OCTAVE, bins_per_octave=BINS_PER_OCTAVE)
    #c = librosa.stft(wav, hop_length=HOP_LENGTH, n_fft=N_FFT)
    return c

def inv_ft(ft):
    return librosa.icqt(ft, hop_length=HOP_LENGTH, fmin=FMIN, bins_per_octave=BINS_PER_OCTAVE)
    #return librosa.istft(ft, hop_length=HOP_LENGTH)
    

In [4]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Clean\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
fft = get_ft(wav)
print(fft.shape)

(768, 477)


In [5]:
# Creating data from clip wave file
# Need to get a subset of the memory mapped file to work on

# Frame output is (samples, bins, window size, layers), where image is depth 2 for real & imaginary
def clip_frames(file, frames_view):
    wav, rate = librosa.core.load(file)
    frame_lead = (WINDOW_SIZE-1)//2  # The amount before and after the target column
    fft = get_ft(wav) # organized as bins, frames
    bins = fft.shape[0]
    max_frames = min(fft.shape[1], frames_view.shape[0])
    pad = np.zeros((bins,frame_lead))
    fft = np.concatenate([pad, fft, pad], axis=-1)
    for i in range(0, max_frames):
        frames_view[i,:,:,0] = abs(fft[:,i:i+WINDOW_SIZE])

# output is array (samples, double bin length made up of real then imag)
# Targets output is array (samples, double bin length made up of real then imag)
def clip_targets(file, targets_view):
    wav, rate = librosa.core.load(file)
    frame_lead = (WINDOW_SIZE-1)//2  # The amount before and after the target column
    fft = get_ft(wav)
    bins = fft.shape[0]
    for i in range(0, min(fft.shape[1], targets_view.shape[0])) :
        targets_view[i,:] = abs(fft[:,i:i+1]).flatten()
    return fft.shape[1]

In [6]:
def frames_file(data_path, max_samples):
    frames_file = data_path + "\\frames-" + str(max_samples)
    filename = os.fsdecode(frames_file)
    return filename

def targets_file(data_path, max_samples):
    targets_file = data_path + "\\targets-" + str(max_samples)
    filename = os.fsdecode(targets_file)
    return filename

# Iterate over clean & noisy folders to create frames and targets
def create_data(wav_root, data_path, max_samples = 10000):
    clean_dir = wav_root + "\\Clean\\"
    noisy_dir = wav_root + "\\Noisy\\"
    sample_index = 0
    frames = np.memmap(frames_file(data_path, max_samples), mode='w+', dtype=DTYPE, shape=(max_samples,FFT_BINS,WINDOW_SIZE,1))
    targets = np.memmap(targets_file(data_path, max_samples), mode='w+', dtype=DTYPE, shape=(max_samples,FFT_BINS))
    #frames = np.empty(dtype=DTYPE, shape=(max_samples,FFT_BINS,WINDOW_SIZE,1))
    #targets = np.empty(dtype=DTYPE, shape=(max_samples,FFT_BINS))
    file_list = os.listdir(clean_dir)
    file_index = 0
    while (sample_index < max_samples) and (file_index < len(file_list)) :
        file = file_list[file_index]
        filename = os.fsdecode(file)
        clip_frames(noisy_dir + file, frames[sample_index:,:,:,:])
        sample_increment = clip_targets(clean_dir + file, targets[sample_index:,:])
        sample_index += sample_increment
        file_index += 1
    print("Reached sample # " + str(min(sample_index, max_samples)))
    return frames, targets


In [8]:
# Training data comes in at ~ 4 * 85K samples total (changed hop size so this will have changed)
f, t = create_data("Assets\\DataShareArchive\\Test", "f:\\Audiodata", max_samples=10000)

Reached sample # 10000


In [8]:
print(t.shape, f.shape)
i = 2
print(f[i,0, 12:13, 0])
print(t[i,0])

(50000, 768) (50000, 768, 25, 1)
[0.00490149]
0.022915643


In [9]:
for i in range(0,2):
    for j in range(0,2):
        if f[i, j, 12, 0] != t[i,j]:
            print ("oh no i ", i, ", j ", j, "   ", )
            print(f[i, j, 12, 0], " != ", t[i,j], "   diff ", f[i, j, 12, 0] - t[i,j])
print("done")
print(i)

oh no i  0 , j  0    
0.004721614  !=  0.022947581    diff  -0.018225968
oh no i  0 , j  1    
0.03477435  !=  0.021355934    diff  0.013418414
oh no i  1 , j  0    
0.0047909436  !=  0.022935083    diff  -0.01814414
oh no i  1 , j  1    
0.034678873  !=  0.021360101    diff  0.0133187715
done
1
